In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DateType, DoubleType
from datetime import datetime, timedelta

In [2]:
spark = (
    SparkSession.builder.appName('Transactions Preprocess')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)


your 131072x1 screen size is bogus. expect trouble
24/09/12 17:03:53 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/12 17:03:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/12 17:03:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
transactions = spark.read.parquet(".././data/transactions/total_transactions")
transactions.show(truncate = False)

+-------+------------+------------------+------------------------------------+--------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|
+-------+------------+------------------+------------------------------------+--------------+
|14935  |79417999332 |136.06570809815838|23acbb7b-cf98-4580-9775-86b8e0a2bd88|2021-11-26    |
|1      |46451548968 |72.61581642788431 |76bab304-fa2d-4004-8179-8638b56a873e|2021-11-26    |
|14936  |89518629617 |3.0783487174439297|a2ae446a-2959-41c4-81fd-a30c1efbde0c|2021-11-26    |
|1      |49167531725 |51.58228625503599 |7080c274-17f7-4ccf-be22-1d33a45f7f81|2021-11-26    |
|14936  |31101120643 |25.228114942417797|8e301c0f-06ab-45cb-a573-47c8e9b95423|2021-11-26    |
|2      |67978471888 |691.5028234458998 |0380e9ad-b0e8-4203-a74b-e4ad3504d643|2021-11-26    |
|14936  |60956456424 |102.13952056640888|5ac3da9c-5147-4524-8efb-0d312cfe6748|2021-11-26    |
|2      |47644196714 |644.5220654863093 |4e368e44-86f8-4dee-

In [4]:
transactions.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [5]:
transactions = transactions.withColumn('user_id',  F.col('user_id').cast(StringType()))

In [6]:
transactions = transactions.withColumn('merchant_abn',  F.col('merchant_abn').cast(StringType()))

In [7]:
transactions.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



Check user_id

In [8]:
consumer_details = spark.read.parquet(".././data/curated/consumer_details")
consumer_details.show(truncate = False)

+-------+-----------+
|user_id|consumer_id|
+-------+-----------+
|1      |1195503    |
|2      |179208     |
|3      |1194530    |
|4      |154128     |
|5      |712975     |
|6      |407340     |
|7      |511685     |
|8      |448088     |
|9      |650435     |
|10     |1058499    |
|11     |428325     |
|12     |1494640    |
|13     |1146717    |
|14     |1343547    |
|15     |1463076    |
|16     |1356405    |
|17     |1331093    |
|18     |80965      |
|19     |1226530    |
|20     |1390367    |
+-------+-----------+
only showing top 20 rows



In [9]:
userid = consumer_details.select(F.col('user_id')).distinct()
userid = userid.toPandas()
userid = userid['user_id'].tolist()

In [10]:
transacid = transactions.select(F.col('user_id')).distinct()
transacid = transacid.toPandas()
transacid = transacid['user_id'].tolist()

In [11]:
transactions.count()

14195505

In [12]:
print(len(set(transacid).difference(userid)))

0


In [13]:
print(set(transacid).difference(userid))

set()


Check merchant_abn

In [14]:
tbl_merchants = spark.read.parquet(".././data/curated/tbl_merchants")
tbl_merchants.show(truncate = False)


+------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------+---------------------------------------------------------------------------------------+-----+-----+
|merchant_name                       |tags                                                                                                             |merchant_abn|tags1                                                                                  |tags2|tags3|
+------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------+---------------------------------------------------------------------------------------+-----+-----+
|Felis Limited                       |[[furniture, home furnishings and equipment shops, and manufacturers, except appliances], [e], [take rate: 0.18]]|10023283211 |[furniture, home furnishings and equi

In [15]:
tbl_merchants = tbl_merchants.withColumn('merchant_abn',F.col('merchant_abn').cast(StringType()) ) 

In [16]:
merchants_abn = tbl_merchants.select(F.col('merchant_abn')).distinct()
merchants_abn = merchants_abn.toPandas()
merchants_abn = merchants_abn['merchant_abn'].tolist()

In [17]:
transacabn = transactions.select(F.col('merchant_abn')).distinct()
transacabn = transacabn.toPandas()
transacabn = transacabn['merchant_abn'].tolist()

In [18]:
transactions.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [19]:
tbl_merchants.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- tags1: string (nullable = true)
 |-- tags2: string (nullable = true)
 |-- tags3: double (nullable = true)



In [20]:
tbl_merchants.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- tags1: string (nullable = true)
 |-- tags2: string (nullable = true)
 |-- tags3: double (nullable = true)



In [21]:
print(len(set(transacabn).difference(merchants_abn)))

396


In [22]:
print(set(transacabn).difference(merchants_abn))

{'87901668493', '86398826270', '40284000376', '22119749782', '83316055023', '24786876262', '87654295342', '32649670245', '33286410763', '71762159356', '85194491864', '61333486380', '73533145785', '53714166188', '60427964172', '73683476429', '88020276364', '58480763378', '13226823813', '13289167958', '66782478748', '22614407882', '74519830790', '11240426404', '63072262874', '80664157260', '59299206461', '51914572250', '63344521351', '95390063681', '87816203170', '68102957262', '89211610270', '25569327276', '53714470577', '84500933183', '17109287252', '41425032366', '15073921424', '63939052570', '11137507330', '31761538646', '98770741241', '56503889644', '98035030376', '90217817387', '56703883889', '19109035261', '23495610951', '11358147682', '55461936812', '72519488277', '68395659657', '50794104497', '70893541900', '66591506230', '33621236826', '26189928515', '97396701267', '90635623930', '42211422632', '71836374493', '87802246756', '74448581190', '87300469819', '30984752802', '96836546

In [23]:
lst = list(set(transacabn).difference(merchants_abn))

In [24]:
dct = {'merchants_abn' : lst}
df = pd.DataFrame(dct)
     
# saving the dataframe
df.to_csv('missingabn.csv')

In [25]:
tbl_merchants.filter(F.col('merchant_abn') == '29836312398').show()

+-------------+----+------------+-----+-----+-----+
|merchant_name|tags|merchant_abn|tags1|tags2|tags3|
+-------------+----+------------+-----+-----+-----+
+-------------+----+------------+-----+-----+-----+



In [26]:
transactions.filter(F.col('merchant_abn') == '29836312398').show()

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|   8571| 29836312398| 2945.505025734725|b627ed8d-262b-4cd...|    2021-12-11|
|  12517| 29836312398|1543.1517906741005|e32c9ad9-ff31-411...|    2022-10-23|
|   1620| 29836312398|   6258.0924425422|fc4c3ccd-dd22-44a...|    2022-10-26|
|  12615| 29836312398|3813.1665926469286|9e572064-6fde-416...|    2022-08-23|
|  21939| 29836312398| 3899.466796707913|00d9d0d2-7b23-403...|    2022-05-29|
|     69| 29836312398| 4411.945163042472|14f63cf5-d8a0-48f...|    2022-07-28|
|  16922| 29836312398|3448.2403740176865|a38da7f6-4da4-4a3...|    2022-06-03|
|   4687| 29836312398|3807.7129245420906|989ac26e-1e23-424...|    2022-06-07|
|  12991| 29836312398| 2072.397078128837|0368c646-6390-4d8...|    2021-11-07|
|  19145| 29836312398| 831.8509457977442|03816e55-0beb-4f6...|  

check dollar value

In [27]:
transactions.filter(F.col('dollar_value') < 0).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



check null values

In [28]:
transactions.filter(F.isnull('dollar_value')).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



In [29]:
transactions.filter(F.isnull('order_id')).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



In [30]:
transactions.filter(F.isnull('user_id')).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



In [31]:
transactions.filter(F.isnull('order_datetime')).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



In [32]:
transactions.filter(F.isnull('merchant_abn')).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



In [33]:
transactions.filter(F.isnan('dollar_value')).show()

+-------+------------+------------+--------+--------------+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|
+-------+------------+------------+--------+--------------+
+-------+------------+------------+--------+--------------+



In [34]:
lst

['87901668493',
 '86398826270',
 '40284000376',
 '22119749782',
 '83316055023',
 '24786876262',
 '87654295342',
 '32649670245',
 '33286410763',
 '71762159356',
 '85194491864',
 '61333486380',
 '73533145785',
 '53714166188',
 '60427964172',
 '73683476429',
 '88020276364',
 '58480763378',
 '13226823813',
 '13289167958',
 '66782478748',
 '22614407882',
 '74519830790',
 '11240426404',
 '63072262874',
 '80664157260',
 '59299206461',
 '51914572250',
 '63344521351',
 '95390063681',
 '87816203170',
 '68102957262',
 '89211610270',
 '25569327276',
 '53714470577',
 '84500933183',
 '17109287252',
 '41425032366',
 '15073921424',
 '63939052570',
 '11137507330',
 '31761538646',
 '98770741241',
 '56503889644',
 '98035030376',
 '90217817387',
 '56703883889',
 '19109035261',
 '23495610951',
 '11358147682',
 '55461936812',
 '72519488277',
 '68395659657',
 '50794104497',
 '70893541900',
 '66591506230',
 '33621236826',
 '26189928515',
 '97396701267',
 '90635623930',
 '42211422632',
 '71836374493',
 '878022

In [35]:
transactions.filter(F.col('merchant_abn') != '29836312398').count()

14195484

In [36]:
transactions = transactions.filter(F.col('merchant_abn') != '29836312398')

In [37]:
transactions.count()

14195484

In [38]:
transactions = transactions.filter(~transactions['merchant_abn'].isin(lst))


In [39]:
transactions.count()

13614675

In [40]:
transacabn1 = transactions.select(F.col('merchant_abn')).distinct()
transacabn1 = transacabn1.toPandas()
transacabn1 = transacabn1['merchant_abn'].tolist()

In [41]:
transactions.count()

13614675

In [42]:
print(len(set(transacabn1).difference(merchants_abn)))

0


In [44]:
transactions.count()

13614675

In [45]:
transactions.write.mode('overwrite').parquet('.././data/curated/total_transactions')